<h1>Implementing LASSO using coordinate descent</h1>

In [5]:
import pandas as pd
import numpy as np
import math


We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

In [12]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = int(1) # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe=data_sframe[features]
    # this will convert the features_sframe into a numpy matrix:
    features_matrix = np.array(features_sframe)
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray=data_sframe[output]
    # this will convert the SArray into a numpy array:
    output_array = np.array(output_sarray)
    return(features_matrix, output_array)

def predict_output(feature_matrix, weights):
    predictions =np.dot(feature_matrix,weights)
    #predictions = x_arr.sum(axis = 1)
    return predictions

In [2]:

def normalize_features(features_matrix):
    norms = np.linalg.norm(features_matrix, axis=0)
    normalized_features = features_matrix / norms
    return (normalized_features, norms)


In [3]:
import pandas as pd
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


In [6]:
feature, price = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

In [8]:
feature, norms = normalize_features(feature)

We assign some random set of initial 

*   List item
*   List item

weights and inspect the values of `ro[i]`:

In [14]:
weights = np.array([1., 4., 1.])
prediction = predict_output(feature, weights)
prediction

array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
       0.02289873])

Compute the values of ro[i] for each feature in this simple model, using the formula given above, using the formula:

ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
Hint: You can get a Numpy vector for feature_i using:

simple_feature_matrix[:,i]


In [16]:
output=price
ro = [0] * feature.shape[1]
for i in range(feature.shape[1]):
    ro[i] = np.dot(feature[:,i], output - prediction + weights[i] * feature[:,i])

In [17]:
ro

[79400300.01452291, 87939470.82325175, 80966698.66623946]

QUIZ QUESTION

Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the corresponding weight w[i] is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to take a step in that coordinate?

In [18]:
def test_l1(feature_matrix, output, weights, l1_penalty):
    prediction = predict_output(feature_matrix, weights)
    for i in range(3):
        ro_i = np.dot(feature_matrix[:,i], output - prediction + weights[i] * feature_matrix[:,i])
        if ro_i < -l1_penalty/2.:
            new_weight_i = ro_i + l1_penalty / 2
        elif ro_i > l1_penalty/2.:
            new_weight_i = ro_i - l1_penalty / 2
        else:
            new_weight_i = 0.
        print(new_weight_i)

In [19]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
weights = np.array([1., 4., 1.])

In [20]:
test_l1_list = [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]

for l1 in test_l1_list:
    print(l1)
    test_l1(simple_feature_matrix, output, weights, l1)
    print

140000000.0
9400300.01452291
17939470.823251754
10966698.666239455
164000000.0
0.0
5939470.823251754
0.0
173000000.0
0.0
1439470.823251754
0.0
190000000.0
0.0
0.0
0.0
230000000.0
0.0
0.0
0.0


So we can say that ro[i] quantifies the significance of the i-th feature: the larger ro[i] is, the more likely it is for the i-th feature to be retained.

<h1>Single Coordinate Descent Step</h1>

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [22]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot(feature_matrix[:,i], output - prediction + weights[i] * feature_matrix[:,i])

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty / 2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty / 2
    else:
        new_weight_i = 0.
    
    return(new_weight_i)

In [23]:
# should print 0.425558846691
import math

lasso = lasso_coordinate_descent_step(1,
                                      np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                      np.array([1., 1.]), 
                                      np.array([1., 4.]),
                                      0.1)

print(lasso)

0.4255588466910251


In [72]:
!pip install tqdm 
import tqdm as tq

# Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

In [73]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [80]:
i = 50
pbar = tqdm(total=1000000)
while i <= 1000000:
    pbar.update(1)
    i +=1
pbar.close()

<ipython-input-80-9d4fccc3cdec>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  pbar = tqdm(total=1000000)


In [101]:
import time

time.time()

1609766350.734688

In [104]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.array(initial_weights)
    it=0
    t=0
    while True:
        it=it+1
        if it%10==0:
            t=time.time()-t
            print(it,t)
        if it>500:
            break
        max_change = None
        #pbar = tqdm(total=feature_matrix.shape[1])
        for i in range(feature_matrix.shape[1]):
            new_weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change = abs(new_weight - weights[i])
            if not max_change or max_change < change:
                max_change = change
            weights[i] = new_weight
        #pbar.close()
        if max_change < tolerance:
            break
    print(it)
    
    return(weights)

In [85]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [86]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [87]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, 
                                            output,
                                            initial_weights, 
                                            l1_penalty, 
                                            tolerance)

print(weights)

93
[21624997.95951909 63157247.20788956        0.        ]


In [88]:
predictions = predict_output(normalized_simple_feature_matrix, weights)
RSS = sum([(predictions[i] - output[i]) ** 2 for i in range(len(predictions))])
print(RSS)

1630492476715379.8


# Evaluating LASSO fit with more features


In [89]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [90]:
train_data = pd.read_csv('kc_house_train_data.csv', dtype = dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype = dtype_dict)
### Let us consider the following set of features.
more_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 
                'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

train_data

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17379,7936000429,20150326T000000,1007500.0,4.0,3.50,3510.0,7200,2.0,0,0,...,9,2600,910,2009,0,98136,47.5537,-122.398,2050.0,6200.0
17380,2997800021,20150219T000000,475000.0,3.0,2.50,1310.0,1294,2.0,0,0,...,8,1180,130,2008,0,98116,47.5773,-122.409,1330.0,1265.0
17381,0263000018,20140521T000000,360000.0,3.0,2.50,1530.0,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530.0,1509.0
17382,0291310100,20150116T000000,400000.0,3.0,2.50,1600.0,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410.0,1287.0


In [91]:
(feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
(normalized_feature_matrix, norms) = normalize_features(feature_matrix)

In [92]:
initial_weights=np.array([0.] * (len(all_features)+1))
weights1e7=[]
weights_1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, 
                                                output,
                                                initial_weights, 
                                                l1_penalty=1e7, 
                                                tolerance=1)

zip(['constant']+all_features, weights_1e7)
weights1e7_dict = {}
for feat, coef in zip(all_features,weights1e7):
    weights1e7_dict[feat] = coef
weights1e7_dict

93


{}

In [95]:
### l1_penalty=1e7

initial_weights1 = [0.]*(len(more_features)+1)
l1_penalty = 1e7
tolerance = 1.0

weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix,
                                               output,
                                               initial_weights1, 
                                               l1_penalty,
                                               tolerance)
weights1e7_dict = {}
for feat, coef in zip(more_features, weights1e7):
    weights1e7_dict[feat] = coef

weights1e7_dict



93


{'bedrooms': 24429600.23440313,
 'bathrooms': 0.0,
 'sqft_living': 0.0,
 'sqft_lot': 48389174.77154894,
 'floors': 0.0,
 'waterfront': 0.0,
 'view': 3317511.214921655,
 'condition': 7329961.811714258,
 'grade': 0.0,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': 0.0,
 'yr_renovated': 0.0}

In [96]:
weights1e7_dict


{'bedrooms': 24429600.23440313,
 'bathrooms': 0.0,
 'sqft_living': 0.0,
 'sqft_lot': 48389174.77154894,
 'floors': 0.0,
 'waterfront': 0.0,
 'view': 3317511.214921655,
 'condition': 7329961.811714258,
 'grade': 0.0,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': 0.0,
 'yr_renovated': 0.0}

In [62]:
### l1_penalty=1e7

initial_weights1 = [0.]*(len(more_features)+1)
l1_penalty = 1e8
tolerance = 1.0

weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix,
                                               output,
                                               initial_weights1, 
                                               l1_penalty,
                                               tolerance)
weights1e8_dict = {}
for feat, coef in zip(more_features, weights1e8):
    weights1e8_dict[feat] = coef

weights1e8_dict



2


{'bedrooms': 71114625.71488702,
 'bathrooms': 0.0,
 'sqft_living': 0.0,
 'sqft_lot': 0.0,
 'floors': 0.0,
 'waterfront': 0.0,
 'view': 0.0,
 'condition': 0.0,
 'grade': 0.0,
 'sqft_above': 0.0,
 'sqft_basement': 0.0,
 'yr_built': 0.0,
 'yr_renovated': 0.0}

In [105]:
### l1_penalty=1e7

initial_weights1 = [0.]*(len(more_features)+1)
l1_penalty = 1e6
tolerance = 1.0

weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix,
                                               output,
                                               initial_weights1, 
                                               l1_penalty,
                                               tolerance)
weights1e4_dict = {}
for feat, coef in zip(more_features, weights1e4):
    weights1e4_dict[feat] = coef

weights1e4_dict



10 1609766559.5163014
20 3.944967031478882
30 1609766562.8693807
40 8.10973334312439
50 1609766566.526276
60 12.098939657211304
70 1609766570.4154131
80 15.85322093963623
90 1609766574.205954
100 20.073033332824707
110 1609766578.2850473
120 24.125747442245483
130 1609766582.3587372
140 27.718189477920532
150 1609766586.864118
160 31.72600030899048
170 1609766590.9543028
180 35.37572193145752
190 1609766595.0762997
200 39.60161232948303
210 1609766599.891108
220 44.22277307510376
230 1609766604.465901
240 48.84816813468933
250 1609766608.9344907
260 53.20852184295654
270 1609766613.0549874
280 57.33002018928528
290 1609766617.3964584
300 61.43255281448364
310 1609766621.0412476
320 65.46727156639099
330 1609766625.340293
340 69.86108064651489
350 1609766629.51665
360 74.20752549171448
370 1609766633.1694558
380 78.26222372055054
390 1609766637.1458676
400 82.40318441390991
410 1609766641.317761
420 86.02052855491638
430 1609766645.5001018
440 89.94009327888489
450 1609766649.337359
460

{'bedrooms': -53832332.57416552,
 'bathrooms': -3745023.632939564,
 'sqft_living': 0.0,
 'sqft_lot': 56959129.180157885,
 'floors': -701896.8335566944,
 'waterfront': 0.0,
 'view': 6469485.37848466,
 'condition': 6758531.17913718,
 'grade': 7950079.851176834,
 'sqft_above': 65748355.398531616,
 'sqft_basement': 0.0,
 'yr_built': 1389314.1245435392,
 'yr_renovated': 0.0}

In [65]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

To check your results, if you call normalized_weights1e7 the normalized version of weights1e7, then:

print normalized_weights1e7[3]
should return 161.31745624837794.

In [108]:
weights1e4_normalized = weights1e4 / norms
weights1e7_normalized = weights1e7 / norms
weights1e8_normalized = weights1e8 / norms

print(weights1e7_normalized[3])

161.31745764611756


## Evaluating each of the learned models on the test data

In [109]:
predictions = predict_output(test_feature_matrix, weights1e4_normalized)
RSS = sum([(predictions[i] - test_output[i]) ** 2 for i in range(len(predictions))])
print(RSS)

218712012075843.0


In [68]:
predictions = predict_output(test_feature_matrix, weights1e7_normalized)
RSS = sum([(predictions[i] - test_output[i]) ** 2 for i in range(len(predictions))])
print(RSS)

275962075920366.72


In [69]:
predictions = predict_output(test_feature_matrix, weights1e8_normalized)
RSS = sum([(predictions[i] - test_output[i]) ** 2 for i in range(len(predictions))])
print(RSS)

537166151497321.94
